# Atlas OdF 2021-2024

In [ ]:
import sys

import pathlib as pl

import pandas as pd
import numpy as np

In [ ]:
sys.version

# Reste à faire sur les mailles OdF

In [ ]:
# Ordre systématique des noms d'espèce
dfEspRef = pd.read_excel('donnees/EspecesRef.ods', usecols=['NomFrancais', 'CodeClassif']).drop_duplicates()

In [ ]:
# Liste d'espèces à trouver au minimum (en gros) sur les carrés Combraille / Limagne / Bourbonnais
espCibles = ['Grèbe castagneux',
             'Canard colvert',
             'Bondrée apivore', 'Milan noir', 'Milan royal', 'Circaète Jean-le-Blanc',
             'Busard Saint-Martin', 'Busard cendré', 'Autour des palombes', "Epervier d'Europe",
             'Buse variable', 'Faucon crécerelle', 'Faucon hobereau',
             'Perdrix rouge', 'Perdrix grise', 'Caille des blés', 'Faisan de Colchide',
             "Râle d'eau", "Gallinule poule-d'eau", 'Foulque macroule',
             'Oedicnème criard', 'Petit gravelot', 'Vanneau huppé', 'Courlis cendré',
             'Pigeon colombin', 'Pigeon ramier', 'Tourterelle turque', 'Tourterelle des bois',
             'Coucou gris',
             "Grand-duc d'Europe", "Chevêche d'Athéna", 'Chouette hulotte', 'Hibou moyen-duc',
             "Engoulevent d'Europe",
             'Martinet noir', "Martin-pêcheur d'Europe", "Guêpier d'Europe", 'Huppe fasciée',
             'Torcol fourmilier', 'Pic vert', 'Pic noir', 'Pic épeiche', 'Pic mar', 'Pic épeichette',
             'Alouette lulu', 'Alouette des champs',
             'Hirondelle rustique', 'Hirondelle de fenêtre',
             'Pipit des arbres', 'Bergeronnette printanière', 'Bergeronnette des ruisseaux', 'Bergeronnette grise',
             'Cincle plongeur', 'Troglodyte mignon', 'Accenteur mouchet',
             'Rougegorge familier', 'Rossignol philomèle', 'Rougequeue noir', 'Rougequeue à front blanc', 'Tarier pâtre',
             'Merle noir', 'Grive musicienne', 'Grive draine',
             'Cisticole des joncs', 'Rousserolle effarvatte', 'Hypolaïs polyglotte',
             'Fauvette grisette', 'Fauvette des jardins', 'Fauvette à tête noire',
             'Pouillot de Bonelli', 'Pouillot véloce', 'Roitelet huppé', 'Roitelet à triple bandeau',
             'Gobemouche gris',
             'Mésange à longue queue', 'Mésange nonnette', 'Mésange huppée', 'Mésange bleue', 'Mésange charbonnière',
             'Sittelle torchepot', 'Grimpereau des jardins',
             "Loriot d'Europe", 'Pie-grièche écorcheur',
             'Geai des chênes', 'Pie bavarde', 'Choucas des tours', 'Corneille noire', 'Corbeau freux',
             'Etourneau sansonnet',
             'Moineau domestique', 'Moineau friquet',
             'Pinson des arbres', 'Serin cini', "Verdier d'Europe", 'Chardonneret élégant', 'Linotte mélodieuse',
             'Bouvreuil pivoine', 'Grosbec casse-noyaux',
             'Bruant jaune', 'Bruant zizi', 'Bruant des roseaux', 'Bruant proyer']
len(espCibles)

In [ ]:
# Maille fictive cible pour la liste des espèces ... ciblées
dfMailleCible = dfEspRef[dfEspRef.NomFrancais.isin(espCibles)].rename(columns=dict(NomFrancais='Espèce')).copy()
dfMailleCible['Rien'] = np.nan
dfMailleCible.set_index(['CodeClassif', 'Espèce'], inplace=True)
dfMailleCible.columns = pd.MultiIndex.from_tuples([('Cible', 'Rien')])
dfMailleCible

In [ ]:
annee = 2021

In [ ]:
dossier = pl.Path('donnees/atlas2124')

In [ ]:
#codesMailles = [f'E{lon:03d}N{lat:03d}' for lon in range(69, 74) for lat in range(653, 657)] # + ['E071N651']

In [ ]:
mailles = \
{
  'E070N654': 'Combronde',
  'E071N654': 'Aigueperse',
  'E070N655': 'Ebreuil',
  'E071N655': 'Gannat',

  'E071N653': 'Ennezat',
  'E072N655': 'Biozat',
  
  'E070N656': 'Bellenaves',
  'E071N656': 'Jenzat',

  'E072N653': 'Maringues',
  'E072N654': 'Randan',
  'E072N656': 'Broût-Vernet',

  'E069N653': 'Pulvérières',
  'E069N654': 'Saint-Angel',
  'E069N655': 'Menat',
  'E069N656': 'Echassières',
    
  #'E070N653': 'Chatelguyon',
    
  #'E071N651': 'La Roche Noire'
}

In [ ]:
def corrigerNomFrancais(esp):
    
    if 'biset' in esp:
        esp = 'Pigeon biset'  # domestique
    elif esp.startswith('Grand') or esp.startswith('Petit'):
        esp = esp[:6] + esp[6].lower() + esp[7:]  # Grand / Petit <machin>
    elif 'É' in esp:
        esp = esp.replace('É', 'E')
    elif esp.startswith('Bergeronnette centratlantique'):
        esp = 'Bergeronnette centratlantique'
    
    return esp

In [ ]:
especesAIgnorer = ['Bergeronnette printanière (M.f.flava)']

In [ ]:
ldfMailles = list()
for codMail, nomMail in mailles.items():

    chemFic = dossier / f'export_atlas_{codMail}.xlsx'
    print(codMail, ':', nomMail or '<pas de nom>', end=', ')
    if not chemFic.is_file():
        print('pas de fichier')
        continue
    
    mTime = pd.Timestamp.fromtimestamp(chemFic.stat().st_mtime).strftime('%d/%m %H:%M')
    print('ddm=' + mTime, end=', ')
    
    dfMaille = pd.read_excel(chemFic, header=None, usecols=[0, 1, 2, 3], names=['Poss', 'Prob', 'Cert', 'Espèce'])

    assert dfMaille.loc[0].Prob == codMail
    assert int(dfMaille.loc[1].Prob) == annee
    pressObs = dfMaille.loc[2].Prob
    print('po={}h'.format(int(pressObs)), end=', ')

    dfMaille.drop([0, 1, 2, 3], inplace=True)

    dfMaille['Espèce'] = dfMaille['Espèce'].apply(corrigerNomFrancais)
    dfMaille.drop(dfMaille[dfMaille['Espèce'].isin(especesAIgnorer)].index, inplace=True)
    
    dfMaille = dfMaille.join(dfEspRef.set_index('NomFrancais'), on='Espèce')
    assert not any(dfMaille.CodeClassif.isnull())
    dfMaille.CodeClassif = dfMaille.CodeClassif.astype(int)

    dfMaille.set_index(['CodeClassif', 'Espèce'], inplace=True)
    
    dfMaille.columns = pd.MultiIndex.from_tuples([('{} {} {}'.format(nomMail, codMail, mTime), col)
                                                  for col in dfMaille.columns])
    
    ldfMailles.append(dfMaille)
    
    #break
    
    print()  
    
ldfMailles.append(dfMailleCible)
dfMailles = pd.concat(ldfMailles, axis='columns').sort_index().reset_index()
dfMailles.drop(columns=['CodeClassif', 'Cible'], inplace=True, level=0)

dfMailles

In [ ]:
def syntheseIndiceRepro(s):
    return 'Certaine' if s.Cert > 0 else 'Probable' if s.Prob > 0 else 'Possible' if s.Poss > 0 else ''

dfMaillesSynth = dfMailles[['Espèce']].copy()
dfMaillesSynth.columns = ['Espèce']  # Down to 1 level
for mail in [col for col in dfMailles.columns.get_level_values(0).unique() if col != 'Espèce']:
    dfMaillesSynth[mail] = dfMailles[mail].apply(syntheseIndiceRepro, axis='columns')
dfMaillesSynth.set_index('Espèce', inplace=True)

dfMaillesSynth

In [ ]:
def couleurIndice(indice):
    color = '#cbef8c' if indice == 'Certaine' else '#f9da56' if indice == 'Probable' \
                      else '#fe835a' if indice == 'Possible' else None
    return 'background-color: ' + color if color else None

dfsMaillesSynth = dfMaillesSynth.style.applymap(couleurIndice)
dfsMaillesSynth

In [ ]:
fn = 'Atlas2124-Avancement'

with pd.ExcelWriter(dossier / f'{fn}.xlsx') as xlWrtr:
    dfsMaillesSynth.to_excel(xlWrtr, sheet_name='Synthèse', index=True)
    dfMailles.to_excel(xlWrtr, sheet_name='Détails')

with open(dossier / f'{fn}-synthese.html', 'w') as file:
    file.write(dfsMaillesSynth.render())
dfMailles.to_html(dossier / f'{fn}-details.html', index=False, na_rep='')

# Génération KML pour carrés 1km

In [ ]:
#from lxml import etree
import simplekml as skml # Simple KML generation API
import kmlcircle # Circle generation for KML generation

In [ ]:
squareCenters = \
[
    (46.108257, 3.071173), # Les Plagnots, Ebreuil (centre équestre)
]

In [ ]:
squareWidth = 1000 # m
squareRadius = np.sqrt(2*(squareWidth/2)**2)

In [ ]:
kml = skml.Kml()

In [ ]:
lineStyle = skml.LineStyle(color=skml.Color.red, width=1)

In [ ]:
for lat, long in squareCenters:
    ls = kml.newlinestring(coords=kmlcircle.spoints(long=long, lat=lat, meters=squareRadius, n=4, offset=45))
    ls.linestyle = lineStyle

In [ ]:
kml.save('carres-1km.kml')

# Liste des espèces de passereaux à réviser pour les EPOC

En partant de la liste du rapport de synthèse STOC 2001-2019.

In [ ]:
import tabula

In [ ]:
# Nomenclature des espèces
dfEspRef = pd.read_excel('donnees/EspecesRef.ods', engine='odf')
dfEspRef

In [ ]:
tables = tabula.read_pdf('/perso/documents/Ornithologie/STOCTendancesSpecifiques-2001-2019.pdf',
                         pages = "all", multiple_tables = True)
len(tables)

In [ ]:
dfEspEpoc = pd.concat(tables, ignore_index=True)

dfEspEpoc.columns = dfEspEpoc.iloc[:2].apply(lambda sr: ' '.join(str(s) if isinstance(s, str) else '' for s in sr),
                                             axis='index')
dfEspEpoc.columns = ['Espèce'] + list(dfEspEpoc.columns[1:])

dfEspEpoc.drop([0, 1], inplace=True)

dfEspEpoc.loc[dfEspEpoc['Espèce'] == 'Rousserolle effarvate', 'Espèce'] = 'Rousserolle effarvatte'  # M'enfin !?

dfEspEpoc

In [ ]:
dfEspEpoc = dfEspEpoc.join(dfEspRef[['NomFrancaisBis', 'CodeClassif']].drop_duplicates().set_index('NomFrancaisBis'), on='Espèce')
dfEspEpoc.CodeClassif = dfEspEpoc.CodeClassif.astype(int)
dfEspEpoc

In [ ]:
assert dfEspEpoc.CodeClassif.notnull().all()

In [ ]:
dfEspEpoc.to_excel('donnees/atlas2124/EspecesAReviserEPOC.ods', index=False)